In [1]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go
from dateutil import parser
import datetime as dt

from api.oanda_api import OandaApi
from infrastructure.instrument_collection import instrumentCollection as ic
from db.db import DataDB

In [2]:
database = DataDB()

In [3]:
ic.LoadInstrumentsDB()

In [4]:
pairs = []
for pair, val in ic.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [ ]:
pairs

In [ ]:
data = []
api = OandaApi()
for p in pairs:
    d_temp = api.get_candles_df(pair_name=p, granularity="D", count=400)
    d_temp['gain'] = ((d_temp.mid_c - d_temp.mid_o) / d_temp.mid_c) * 100
    d_temp['pair'] = p
    if '_GBP' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

In [ ]:
candles_df.tail()

In [24]:
calendar_data = database.query_all(DataDB.CALENDAR_COLL)

In [26]:
calendar_data_df = pd.DataFrame.from_dict(calendar_data)

In [ ]:
calendar_data_df.date = calendar_data_df.date.dt.date

In [ ]:
calendar_data_df_uk = calendar_data_df[calendar_data_df.country == "united kingdom"].copy()

In [ ]:
calendar_data_df_uk

In [ ]:
calendar_data_df_uk.info()

In [ ]:
for col in ['actual', 'previous', 'forecast']:
    for sy in ['£', '%', 'B', 'K']:
        calendar_data_df_uk[col] = calendar_data_df_uk[col].str.replace(sy, "")
    calendar_data_df_uk[col].replace('', 0, inplace=True)
    calendar_data_df_uk[col] = calendar_data_df_uk[col].astype(float)